In [1]:
import json
import os
import time
import warnings
import h5py
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import pandas as pd
import pysam
import pyfaidx
import tensorflow as tf
from baskerville import seqnn
from baskerville import gene as bgene
from baskerville import dna
from borzoi_helpers import *
from sklearn.preprocessing import minmax_scale

# Отключаем лишние предупреждения
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


2025-02-19 02:29:03.525097: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 02:29:03.578381: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-19 02:29:03.578441: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-19 02:29:03.578507: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-19 02:29:03.594180: I tensorflow/core/platform/cpu_feature_g

In [33]:
# Пути к файлам
work_path = '/mnt/10tb/home/patsukevichan/borZoi/borzoi'  # путь к вашей рабочей папке
fasta_open = pysam.Fastafile('hg38/assembly/ucsc/hg38.fa')


# Указание на ген ACTRT2 (используем координаты наизусть для примера)
search_gene = 'ACTRT2'
chrom = 'chr1'

# Координаты гена ACTRT2 (примерные для hg38)
start, end = 3_020_000, 3_030_000  # Начало и конец для ACTRT2

# Генерация one-hot последовательности
sequence_one_hot_wt = process_sequence(fasta_open, chrom, start, end)



In [4]:
# Параметры модели и пути к файлам
params_file = f'{work_path}/examples/params_pred.json'  # Путь к файлу параметров модели
targets_file = f'{work_path}/examples/targets_gtex.txt'  # Путь к целям модели
seq_len = 524288  # Размер окна для предсказания
n_reps = 1  # Используем только одну реплику модели
rc = True  # Среднее по реверс-комплементарным предсказаниям

# Чтение параметров модели
with open(params_file) as params_open:
    params = json.load(params_open)
    params_model = params['model']
    params_train = params['train']

# Чтение targets
targets_df = pd.read_csv(targets_file, index_col=0, sep='\t')
target_index = targets_df.index

# Инициализация модели
models = []
for rep_ix in range(n_reps):
    model_file = f"{work_path}/examples/saved_models/f3c{rep_ix}/train/model0_best.h5"  # Путь к файлу модели
    seqnn_model = seqnn.SeqNN(params_model)
    seqnn_model.restore(model_file, 0)
    seqnn_model.build_slice(target_index)
    seqnn_model.build_ensemble(rc, [0])
    models.append(seqnn_model)



2025-02-19 02:29:09.355161: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 23390 MB memory:  -> device: 0, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:01.0, compute capability: 7.0
2025-02-19 02:29:09.355827: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30965 MB memory:  -> device: 1, name: Tesla V100-PCIE-32GB, pci bus id: 0000:21:02.0, compute capability: 7.0


In [43]:
def write_bedgraph_output(predictions, chrom, start, end, output_file):
    """
    Запись предсказаний модели в файл BedGraph.
    """
    print(f"Количество предиктов {len(predictions)}")

    with open(output_file, 'w') as f:
        for i, val in enumerate(predictions):
            pos_start = start + i * 32  # сдвиг с шагом stride (32)
            pos_end = pos_start + 32  # шаг размерности предсказания
            if pos_end > end:
                pos_end = end
            if pos_end <= pos_start:
                break
            f.write(f"{chrom}\t{pos_start}\t{pos_end}\t{float(val[0])}\n")  # Убедимся, что val скаляр
    print(f"Файл {output_file} сохранен.")


In [38]:
def infer_and_save_model(chrom, start, end, models, output_dir):
    """
    Предсказание с использованием модели для указанного гена
    и запись результата в файл bedGraph.
    """
    start_f = ((start + end) / 2) - seq_len / 2  
    end_f = ((start + end) / 2) + seq_len / 2  

    # Получаем one-hot для последовательности
    seq_plus = process_sequence(fasta_open, chrom, start, end)
    
    if seq_plus is None or seq_plus.shape[0] == 0:
        print("Ошибка в получении последовательности!")
        return

    # Предсказания для + и - цепи
    y_plus = predict_tracks(models, seq_plus) 
    print(y_plus.shape)  # вывод размерности предсказаний
    y_plus = y_plus[0, 0, :, :]
    print(y_plus.shape)  # вывод размерности предсказаний

    
    seq_minus = reverse_complement_onehot(seq_plus)
    y_minus = predict_tracks(models, seq_minus)
    y_minus = y_minus[0, 0, :, :]

    # Записываем результат в BedGraph
    output_file_plus = os.path.join(output_dir, f"ACTRT2_{chrom}_st+.bedGraph")
    output_file_minus = os.path.join(output_dir, f"ACTRT2_{chrom}_st-.bedGraph")
    
    write_bedgraph_output(y_plus, chrom, start, end, output_file_plus)
    write_bedgraph_output(y_minus, chrom, start, end, output_file_minus)


In [42]:

def reverse_complement_onehot(seq_onehot):
    """Преобразует one-hot последовательность (L,4) в её реверс-комплемент.
    Предполагается, что:
      индекс 0 = A, 1 = T, 2 = G, 3 = C.
    Результат: A→T, T→A, G→C, C→G.
    """
    rev = np.flip(seq_onehot, axis=0)
    revcomp = rev[:, [1, 0, 3, 2]]
    return revcomp


# Исходная последовательность: ATGC
seq = np.array([[1, 0, 0, 0],  # A
                [0, 1, 0, 0],  # T
                [0, 0, 1, 0],  # G
                [0, 0, 0, 1]]) # C


rev_seq = reverse_complement_onehot(seq)

# Функция для перевода one-hot в строку (при условии, что 0=A,1=T,2=G,3=C)
bases = ['A', 'T', 'G', 'C']
def onehot_to_seq(onehot):
    return ''.join([bases[np.argmax(row)] for row in onehot])

print("Исходная последовательность (в символах):", onehot_to_seq(seq))
print("Реверс-комплементарная последовательность (в символах):", onehot_to_seq(rev_seq))



Исходная последовательность (в символах): ATGC
Реверс-комплементарная последовательность (в символах): GCAT


In [44]:
# Параметры
output_dir = "predicted_expression_by_gene/"
os.makedirs(output_dir, exist_ok=True)

# Запуск инференса для гена ACTRT2
print("Start inference for ACTRT2 gene...")
infer_and_save_model(chrom, start, end, models, output_dir)
print("Done.")


Start inference for ACTRT2 gene...
(1, 1, 16352, 89)
(16352, 89)
16352
Файл predicted_expression_by_gene/ACTRT2_chr1_st+.bedGraph сохранен.
16352
Файл predicted_expression_by_gene/ACTRT2_chr1_st-.bedGraph сохранен.
Done.
